In [135]:
import numpy as np
import array
from keras.datasets import mnist
import keras.utils

In [213]:
(train_data,train_label),(test_data,test_label) = mnist.load_data()
train_label_data_ = keras.utils.to_categorical(train_label, 10)
test_label_data_ = keras.utils.to_categorical(test_label, 10)
train_data = train_data / 255
test_data = test_data / 255
train_data_data_ = train_data.reshape(60000,784)
test_data_data_ = test_data.reshape(10000,784)
# 1. mean = 0, var = 1 baihaar data normalize hiij baigaa heseg
train_label_data = (train_label_data_ - np.mean(train_label_data_))/np.std(train_label_data_)
test_label_data = (test_label_data_ - np.mean(test_label_data_))/np.std(test_label_data_)
train_data_data = (train_data_data_ - np.mean(train_data_data_))/np.std(train_data_data_)
test_data_data = (test_data_data_ - np.mean(test_data_data_))/np.std(test_data_data_)

print('train_label_data_ mean:',np.mean(train_label_data))
print('train_label_data_ var:',np.var(train_label_data))
print('train_label_data_ std:',np.std(train_label_data))

train_label_data_ mean: 6.037553e-09
train_label_data_ var: 0.9999998
train_label_data_ std: 0.9999999


In [229]:
class NeuralNetwork:
    def __init__(self,input_size,hidden_size,batch_size,output_size,learn_rate):
        self.input = input_size
        self.hidden = hidden_size
        self.batch = batch_size
        self.output = output_size
        self.lr = learn_rate
        self.w1 = np.random.normal(loc = 0,scale = 0.1,size=(self.hidden, self.input))
        self.b1 = np.zeros((self.hidden,self.batch))
        self.w2 = np.random.normal(loc = 0,scale = 0.1,size=(self.output, self.hidden))
        self.b2 = np.zeros((self.output,self.batch))
        self.lr = learn_rate
        
        print('w1_shape:',self.w1.shape)
        print('b1_shape:',self.b1.shape)
        print('w2_shape:',self.w2.shape)
        print('b2_shape:',self.b2.shape)
    
    def relu(self,x):
        return np.maximum(0,x)
    # 2.
    # end tsaanaas orj irj baigaa matrix ni output_size x batch_size 
    # hemjeetei baigaa uchiraas axis = 0 uyd jov ajillaj baigaa.
    def softmax(self,k):
        e_x = np.exp(k - np.max(k))
        return e_x / e_x.sum(axis=0)
    
    def step(self,x):
        return np.heaviside(x,1)
    
    def output_formula(self,input_list):
        h = np.add(self.w1@input_list,self.b1)
        a = self.relu(h)
        H = np.add(self.w2@a,self.b2)
        A = self.softmax(H)
        return A
    
    def error_formula(self,output,target):
        loss = []
        for i in range(self.batch):
            ka = -np.log(output[i][target[i]])
            loss.append(ka)
        return np.mean(loss)
    
    def backpropagation(self,input_list,Y):
        
        # ulamjlal avsan baidal
        # dL/dA = ?
        # dL/dH = (dA/dH)*(dL/dA) = (Ak - Yk*)
        # dL/dw2 = a*(Ak - Yk*)
        # dL/db2 = 1*(Ak - Yk*)
        # dL/dw1 = x*step(h)*w2*(Ak - Yk*)
        # dL/db1 = 1*step(h)*w2*(Ak - Yk*)
        # relu'(x) = step(x)
        
        h = np.add(self.w1@input_list,self.b1)
        a = self.relu(h)
        H = np.add(self.w2@a,self.b2)
        A = self.softmax(H)
        A = np.array(A, ndmin=2).T
        
        dH = A - Y
        
        dw2 = a@dH
        dw2 = np.transpose(dw2)
        db2 = dH
        
        step_ = self.step(h)
        step_ = np.transpose(step_)
        dw1 = input_list@step_
        dw1 = np.transpose(dw1)
        dw1 = self.w2@dw1
        dw1 = dH@dw1
        
        # dw1-n shape ni input_list tei adilhan bolood baih ym?
        
        print('dw1 shape:',dw1.shape)
#         print('dw1 value:',dw1)
        
#         self.w2 = self.w2 - dw2*self.lr
        
#         self.w1 = self.w1 - dw1*self.lr

In [230]:
input_size = 784
hidden_size = 32
batch_size = 5
output_size = 10
learn_rate = 0.1
epochs = 1

object1 = NeuralNetwork(input_size,hidden_size,batch_size,output_size,learn_rate)
error_avg = []

for e in range(epochs):
    print('Epoch:',e)
    for i in range(0,500,batch_size):
        batch = train_data_data[i:min(i+batch_size,train_data_data.shape[0]),:]
        label = train_label_data[i:min(i+batch_size,train_label_data.shape[0]),:]
        batch = np.array(batch, ndmin=2).T
        target = train_label[i:i+batch_size]
        output = object1.output_formula(batch)
        output = np.array(output, ndmin=2).T
        error = object1.error_formula(output,target)
        print('error:',':',error)
        error_avg.append(error)
        object1.backpropagation(batch,label)
    print('LOSS AVG:',np.mean(error_avg))

w1_shape: (32, 784)
b1_shape: (32, 5)
w2_shape: (10, 32)
b2_shape: (10, 5)
Epoch: 0
error: : 3.0968247605859083
dw1 shape: (5, 784)
error: : 2.5139298305217714
dw1 shape: (5, 784)
error: : 3.437159349467767
dw1 shape: (5, 784)
error: : 1.9924822414265606
dw1 shape: (5, 784)
error: : 3.0050504371203126
dw1 shape: (5, 784)
error: : 2.4316495712082022
dw1 shape: (5, 784)
error: : 3.061423373930637
dw1 shape: (5, 784)
error: : 2.869893477397819
dw1 shape: (5, 784)
error: : 2.743880201412453
dw1 shape: (5, 784)
error: : 2.6784313001414466
dw1 shape: (5, 784)
error: : 3.3955196749133782
dw1 shape: (5, 784)
error: : 3.409137266677182
dw1 shape: (5, 784)
error: : 2.8953407908386533
dw1 shape: (5, 784)
error: : 2.5978415840544473
dw1 shape: (5, 784)
error: : 2.3603763597968217
dw1 shape: (5, 784)
error: : 1.948411762269783
dw1 shape: (5, 784)
error: : 2.1062727664841185
dw1 shape: (5, 784)
error: : 3.5449216072440755
dw1 shape: (5, 784)
error: : 2.4797479066001493
dw1 shape: (5, 784)
error: : 3